# Advent of code 2021

My first time entering Advent of Code. I'm pretty rusty in Python at the moment, so I have shameless copied the utility functions from Peter Norvieg's [2020 solutions](https://github.com/norvig/pytudes/blob/main/ipynb/Advent-2020.ipynb) so that my attempts will at least start off in a well structured fashion.

Only entering to complete, not to get a fast time.

In [1]:
from __future__  import annotations
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, product, chain, tee
from functools   import lru_cache, reduce
from typing      import Dict, Tuple, Set, List, Iterator, Optional, Union, NamedTuple

import operator
import math
import ast
import sys
import re

Peter Norveigs utilities:

In [2]:
def data(day: int, parser=str, sep='\n') -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    sections = open(f'data/input{day}.txt').read().rstrip().split(sep)
    return [parser(section) for section in sections]
     
def do(day, *answers) -> Dict[int, int]:
    "E.g., do(3) returns {1: day3_1(in3), 2: day3_2(in3)}. Verifies `answers` if given."
    g = globals()
    got = []
    for part in (1, 2):
        fname = f'day{day}_{part}'
        if fname in g: 
            got.append(g[fname](g[f'in{day}']))
            if len(answers) >= part: 
                assert got[-1] == answers[part - 1], (
                    f'{fname}(in{day}) got {got[-1]}; expected {answers[part - 1]}')
    return got

Number = Union[float, int]
Atom = Union[Number, str]
Char = str # Type used to indicate a single character

flatten = chain.from_iterable

def prod(numbers) -> Number:
    "The product of an iterable of numbers." 
    return reduce(operator.mul, numbers, 1)

def atoms(text: str, ignore=r'', sep=None) -> Tuple[Union[int, str]]:
    "Parse text into atoms (numbers or strs), possibly ignoring a regex."
    if ignore:
        text = re.sub(ignore, '', text)
    return tuple(map(atom, text.split(sep)))

def atom(text: str) -> Union[float, int, str]:
    "Parse text into a single float or int or str."
    try:
        val = float(text)
        return round(val) if round(val) == val else val
    except ValueError:
        return text

My utilities

In [3]:
def pairwise(iterable): # Defined in itertools in python 3.10 but I'm in 3.9
    "pairwise('ABCDEFG') --> AB BC CD DE EF FG"
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

### Day 1: Sonar Sweep
Part 1 is to compute the number of times we see an increase between elements. Itertools has a helpful _pairwise_ function that allows comparison between two adjacent elements in a list without making a full copy. Part 2 is to compute a sliding window first.. so I 'adapted' the _pairwise_ function to allow a 3-element sliding window but it's not nicely generalised to different window lengths.

In [4]:
in1: List[int] = data(1, int)

In [5]:
def day1_1(nums):
    return sum(second > first for first,second in pairwise(nums))

In [6]:
def day1_2(nums):
    "Niave extension of itertools.pairwise"
    a, b, c = tee(nums, 3)
    next(b, None)
    next(c, None)
    next(c, None)
    moving_average = [x+y+z for x, y, z in zip(a, b, c)]
    return day1_1(moving_average)

In [7]:
assert day1_1([199, 200, 208, 210, 200, 207, 240, 269, 260, 263]) == 7
assert day1_2([199, 200, 208, 210, 200, 207, 240, 269, 260, 263]) == 5

In [8]:
do(1, 1832, 1858)

[1832, 1858]

### Day 2: Dive!

For part 1 we don't need to worry about the order of the commands (assuming that we don't need to verify that the depth is always sensible). Brute fore approach for part 2.

In [9]:
Instruction = Tuple[str, int] # e.g. ('up', 1)
Program = List[Instruction]

in2: Program = data(2, atoms)

In [10]:
def sum_direction(program, direction):
    return sum(x[1] for x in filter(lambda x: x[0]==direction, program))

assert sum_direction([('down',2),('forward',7),('down',1)],"down") == 3

def day2_1(program):
    "Assumes that the input is valid and we don't have to worry about the submarine trying to go above the waterline"
    horizontal_position = sum_direction(program,"forward")
    depth = sum_direction(program,"down") - sum_direction(program,"up")
    return depth * horizontal_position

In [11]:
def day2_2(program):
    aim, depth, horizontal_position = 0, 0, 0
    for command, v in program:
        if command == "down":
            aim += v
        elif command == "up":
            aim -= v
        elif command == "forward":
            depth += aim * v
            horizontal_position += v
        else:
            raise Exception('Unexpected command %s',command)
    return depth * horizontal_position

In [12]:
do(2, 1484118, 1463827010)

[1484118, 1463827010]

### Day 3: Binary Diagnostic
Full brute force today. For part I I'm sure there must be a better way to unpack the most common element from a _Counter_ and to use the two's complement operator to compute epsilon. In part II, there must be a better way to find the most common element with a default.

I might revisit  these at a later date.

In [13]:
in3: List(int) = data(3, str)

In [14]:
def day3_1(nums):
    nums = [list(x) for x in nums]
    nums = list(map(list, zip(*nums)))
    counters = [Counter(x) for x in nums]
    gamma = "".join(str(x.most_common(1)[0][0]) for x in counters)
    epsilon = "".join(str(x.most_common()[-1][0]) for x in counters)
    return int(gamma,base=2) * int(epsilon, base=2)

def get_rating(nums, mode):
    ix = 0
    while len(nums) > 1 and ix < len(nums[0]):
        counter = Counter(x[ix] for x in nums)
        v = counter.most_common(1)[0] if mode=="most" else counter.most_common()[-1]
        if v[1]*2 == len(nums):
            v = '1' if mode=="most" else '0'
        else:
            v = v[0]
        nums = list(filter(lambda x: x[ix] == v,nums))
        ix += 1
    return nums[0]

def day3_2(nums):
    oxygen = get_rating(nums,mode="most")
    co2 = get_rating(nums,mode="least")
    return int(oxygen,base=2) * int(co2,base=2)

In [15]:
do(3, 2724524, 2775870)

[2724524, 2775870]

### Day 4: Giant Squid
I implemented a BingoBoard class on my first attempt here, but decided this was unnecessary when I revisited it. I think my runtime complexity is _O(NDB^2)_ for N boards, B elements per board and D draws because of the _is_bingo_ function, but the input is not that big.

In [16]:
def parse_bingo_boards(line: str) -> List[int]:
    return list(map(int,re.split("\W+",line.strip())))

in4: List[int] = data(4, parse_bingo_boards, sep='\n\n')

In [17]:
def is_bingo(hits, width = 5):
    out = any(all(hits[i:i+width]) for i in range(0,len(hits),width)) # check for complete rows
    if not out:
        out = any(all(hits[i:len(hits):width]) for i in range(width)) # check for complete columns
    return out

def score_bingo_board(board, draws):
    hits = [False for x in range(len(board))]
    bingo = False
    for i, draw in enumerate(draws):
        if draw in board:
            hits[board.index(draw)] = True
            bingo = is_bingo(hits)
            if bingo: break
    if bingo:
        return (i, draw * sum(x for x, h in zip(board,hits) if not h))
    else:
        return (math.inf, None)

In [18]:
def day4_1(boards):
    scores = [score_bingo_board(x, boards[0]) for x in boards[1:]]
    out = min(scores, key = lambda x: x[0])
    return out[1]

def day4_2(boards):
    scores = [score_bingo_board(x, boards[0]) for x in boards[1:]]
    scores = filter(lambda x: math.isfinite(x[0]), scores)
    out = max(scores, key = lambda x: x[0])
    return out[1]

In [19]:
do(4, 49860, 24628)

[49860, 24628]

### Day 5: Hydrothermal Venture
Part I is to count the number of overlapping points given a set of horizontal / vertical line segments. Pythons _Counter_ again was quite useful to count the number of overlaps. Part II just required an extension to _parse\_line\_segments_ to deal with diagonal lines.

In [20]:
def parse_line_segments(line: str) -> Tuple[int, int, int, int]:  
    a,b,c,d = map(int,re.findall(r'[^->,\s]+',line))
    assert a == c or b == d or (abs(a-c) == abs(b-d)), "Lines must be at 0, 45, 90 degrees"
    return (a,b,c,d)

in5: Tuple[int, int, int, int] = data(5, parse_line_segments)

In [21]:
def is_diagonal(segment):
    a,b,c,d = segment
    return (abs(a-c) == abs(b-d)) and not (a == c or b == d)

def expand_segment(segment):
    a,b,c,d = segment
    if a == c:
        x1, x2 = min(b,d), max(b, d)
        return [(a,x) for x in range(x1,x2+1)]
    elif b == d:
        x1, x2 = min(a,c), max(a, c)
        return [(x,b) for x in range(x1,x2+1)]
    else:
        signX = 1 if c > a else -1
        signY = 1 if d > b else -1
        return [(a+signX*i,b+signY*i) for i in range(abs(a-c)+1)]

In [22]:
def day5_1(segments):
    counter = Counter()
    for segment in segments:
        if not is_diagonal(segment):
            counter.update(expand_segment(segment))
    return sum(counter[x]>1 for x in counter)

def day5_2(segments):
    counter = Counter()
    for segment in segments:
        counter.update(expand_segment(segment))
    return sum(counter[x]>1 for x in counter)

In [23]:
do(5, 5167, 17604)

[5167, 17604]

### Day 6: Lanternfish
A question about modelling the population size of some fish, given that they reproduce every 6 days (or 8 if they are new), which lends itself nicely to recursion and the _lru\_cache_. The grid of cached values should be only 8 * 256 for part II.

In [24]:
in6: List[int] = data(6,int,sep=",")

In [25]:
@lru_cache
def population_size(fish_timer, days):
    step = min(days, fish_timer+1)
    if days == 0:
        out = 1
    elif days >= fish_timer+1:
        out = population_size(6, days - fish_timer - 1) + population_size(8, days - fish_timer - 1)
    else:
        out = population_size(fish_timer - days, 0)
    return out

assert population_size(3,3) == 1
assert population_size(3,4) == 2
assert population_size(3,10) == 2
assert population_size(3,11) == 3
assert population_size(3,13) == 4

In [26]:
def day6_1(fish_timers): return sum(population_size(x,80) for x in fish_timers)

def day6_2(fish_timers): return sum(population_size(x,256) for x in fish_timers)

In [27]:
do(6, 386755, 1732731810807)

[386755, 1732731810807]

### Day 7: The Treachery of Whales
Part I is to compute _min\_p \sum\_i abs(x\_i - p)_ and part II is to compute _min\_p \sum\_i abs(x\_i - p) * (abs(x\_i - p)+1)/2_ for a vector _x_ of initial positions. There is probably a clever way to solve these minimization problems, however the input is not that big so I opted for the brute force approach.

In [28]:
in7: List[int] = data(7,int,sep=",")

In [29]:
def day7_1(positions):
    fuel = math.inf
    for p in range(min(positions),max(positions)+1):
        v = sum(abs(x-p) for x in positions)
        fuel = min(v,fuel)
    return fuel

def day7_2(positions):
    fuel = math.inf
    for p in range(min(positions),max(positions)+1):
        v = sum((abs(x-p)*(abs(x-p)+1))//2 for x in positions)
        fuel = min(v,fuel)
    return fuel

In [30]:
do(7, 351901, 101079875)

[351901, 101079875]

### Day 8: Seven segment search
Labels for a 7 segment display are scrambled, and the task is to decode 4 numbers given an encoded list of numbers 0-9. Part I is relatively straight forward as you are only asked to decode numbers with a unique number of bars. For Part II I ended up with a reasonably verbose tree of if-statements to complete my 'decoder'. 

In [31]:
Display = NamedTuple("Display", [('pattern', List[frozenset]), ('output', List[frozenset])])

def parse_digits(line: str) -> Display:
    line = line.split("|")
    return Display([frozenset(x) for x in re.findall(r'[^\s]+',line[0])],[frozenset(x) for x in re.findall(r'[^\s]+',line[1])])

in8: List[Display] = data(8, parse_digits)

In [32]:
def day8_1(displays: List[Display]) -> int:
    counter = Counter()
    for display in displays:
        counter.update(map(len,display.output))
    return sum(counter[x] for x in [2,4,3,7])

In [33]:
def decode_display(display: Display) -> int:
    lens = {2:1,7:8,4:4,3:7}
    encoder = dict()
    for x in display.pattern:
        if len(x) in lens:
            encoder[lens[len(x)]] = x
    for x in display.pattern:
        if len(x) in lens:
            continue
        elif len(x) == 6 and encoder[4].issubset(x):
            encoder[9] = x
        elif len(x) == 6 and not encoder[1].issubset(x):
            encoder[6] = x
        elif len(x) == 6:
            encoder[0] = x
        elif len(x) == 5 and encoder[7].issubset(x):
            encoder[3] = x
        elif len(x) == 5 and len(x.intersection(encoder[4])) == 3:
            encoder[5] = x
        else:
            encoder[2] = x
    decoder = {v:k for k,v in encoder.items()}
    assert all(x in decoder for x in display.pattern)
    return int("".join([str(decoder[x]) for x in display.output]))


def day8_2(displays: List[Display]) -> int:
    return sum(map(decode_display, displays))


In [34]:
do(8, 387, 986034)

[387, 986034]

### Day 9: Smoke Basin
Given a heatmap find points which are local minima (part I) and the size of the 'basin' around each minima (part II). My solution to part II was inspired by the expanding bubble used in Dijkstra's algorithm. I also found a use for two additional utility functions from Peter Norveigs 2020 solutions; _flatten_ and _prod_.

In [35]:
def parse_ints(line: str) -> List[int]:
    return list(map(int,list(line)))

in9: List[List[int]] = data(9, parse_ints)

In [36]:
def get_neighbours(grid,x,y):
    out = list()
    out += [(i,y) for i in range(max(0,x-1),min(len(grid),x+2)) if not i == x]
    out += [(x,j) for j in range(max(0,y-1),min(len(grid[0]),y+2)) if not j == y]
    return out

def is_minima(grid,x,y):
    return all(grid[x][y] < grid[i][j] for i, j in get_neighbours(grid,x,y))

def day9_1(grid):
    return sum(1+grid[x][y] for x, y in product(range(0,len(grid)),range(0,len(grid[0]))) if is_minima(grid,x,y))

In [37]:
def get_basin_size(grid,x,y):
    in_basin = [[False for i in range(len(grid[0]))] for j in range(len(grid))]
    pool = {(x,y)}
    while pool:
        i, j = pool.pop()
        in_basin[i][j] = True
        for ii, jj in get_neighbours(grid, i, j):
            if not in_basin[ii][jj] and grid[ii][jj] < 9 and grid[ii][jj] > grid[i][j]:
                pool.add((ii,jj))
    return sum(flatten(in_basin))

def day9_2(grid):
    minima = [(x,y) for x, y in product(range(0,len(grid)),range(0,len(grid[0]))) if is_minima(grid,x,y)]
    scores = [get_basin_size(grid,x,y) for x,y in minima]
    scores.sort(reverse=True)
    return prod(scores[:3])


In [38]:
do(9, 600, 987840)

[600, 987840]

### Day 10: Syntax Scoring
Given a string of four different bracket types (eg. _[<>({}){}[([])<>]]_), part I is to write a simple syntax checker to determine if the string is corrupt (brackets are mismatching) and part II is to score the remaining brackets for strings that are incomplete.

I decided to just parse each string from left to right, but you could possibly use recursion here too.

In [39]:
in10: List[str] = data(10) # eg. [<>({}){}[([])<>]]

In [40]:
def score_incomplete(chars):
    points = {')':1,']':2,'}':3,'>':4}
    return sum((5 ** (len(chars)-1-i)) * points[x] for i, x in enumerate(chars))

def check_syntax(line):
    """Parse line from left to right and add opening statements to a heap"""
    open2close = {'{':'}','[':']','(':')','<':'>'}
    points = {')':3,']':57,'}':1197,'>':25137}
    heap = list() # small abuse: not actually a heap but will be used like one
    is_corrupted, score = False, 0
    for i in range(len(line)):
        if line[i] in open2close: 
            heap.append(line[i])
        elif not line[i] == open2close[heap.pop()]:
            is_corrupted, score = True, points[line[i]]
    if not is_corrupted:
        score = score_incomplete([open2close[x] for x in reversed(heap)])
    return (is_corrupted, score)

assert score_incomplete('])}>') == 294
assert check_syntax('[<>[]}') == (True,1197)

In [41]:
def day10_1(input):
    return sum(score for is_corrupted,score in map(check_syntax,input) if is_corrupted)

def day10_2(input):
    scores = sorted([score for is_corrupted,score in map(check_syntax,input) if not is_corrupted])
    return scores[len(scores)//2]

In [42]:
do(10, 388713, 3539961434)

[388713, 3539961434]

### Day 11: Dumbo Octopus
The questions is as follows: consider a 10x10 grid of _Dumbo Octopuses_ which flash once their internal timer reaches a value of ten. The goal is to simulate the behaviour of the grid, and the catch is that when one ocotopus flahes it causes the timer of all neighbouring octupuses to incremement too.

The interaction between octopuses meant I couldn't see any analytic methods to compute this, so I just simulated the system directly. I haven't found a pythonic way to deal with a 2D grid yet - apart from using nympy which seems like overkill for now. Here I flatten the 2D grid into a 1D vector for convenience.

In [43]:
in11: List[List[int]] = data(11, parse_ints)

In [44]:
def get_neighbours(k,n,m):
    row, col = k//n, k%m
    for i in range(max(0, row-1), min(n, row+2)):
        for j in range(max(0, col-1), min(m, col+2)):
            if i == row and j == col: continue
            yield i*m+j

assert [x for x in get_neighbours(11,5,5)] == [5, 6, 7, 10, 12, 15, 16, 17]

def print_grid(grid,n,m):
    lines = ["".join(str(x%10) for x in grid[i*m:(i+1)*m]) for i in range(n)]
    print("\n".join(x for x in lines))
    print("\n")

def simulate_step(grid, n, m):
    grid = [x+1 for x in grid]
    queue = [i for i, x in enumerate(grid) if (x//10) > 0]
    done = set(queue)
    while queue:
        for k in get_neighbours(queue.pop(),n,m):
            grid[k] += 1
            if grid[k]//10 > 0 and not k in done:
                queue.append(k)
                done.add(k)
    for k in done:
        grid[k] = 0
    return grid, len(done)


In [45]:
def day11_1(grid, days = 100):
    n, m = len(grid), len(grid[0])
    grid = list(flatten(grid))
    count = 0
    for i in range(days):
        grid, v = simulate_step(grid, n, m)
        count += v
    return count

def day11_2(grid):
    n, m = len(grid), len(grid[0])
    grid = list(flatten(grid))
    v, i = None, 0
    while not v == len(grid):
        grid, v = simulate_step(grid, n, m)
        i += 1
    return i

In [46]:
do(11, 1642, 320)

[1642, 320]

### Day 12: Passage Pathing
Count the number of valid paths in an undirected graph where no repeat visits (part I) or only 1 repeat visit (part II) is allowed to 'small' nodes. Nodes are small if the key is entirely lower case, and the input graph is constructed such that cycles are not possible (at least, given the constraints from part I/II). I used depth first search to explicitly define all valid paths and then returned the count of these.

This works well enough, but is very slow for part II (10 seconds!).. possibly creating a new list each time I extend an existing path is slow and I could jut maintain the count of unique paths within the dpeth first search.

In [47]:
Edge = Tuple[str,str]

def parse_path(line: str) -> Edge:
    return atoms(line,sep="-")

in12: List(Edge) = data(12, parse_path)
assert all(x in flatten(in12) for x in ['start','end'])

In [48]:
def edges_to_adjacency_table(edges:  List[str]):
    adj = defaultdict(list)
    for edge in edges:
        adj[edge[0]].append(edge[1])
        adj[edge[1]].append(edge[0])
    return adj

def path_is_valid(path: List[str], max_small_cave_repeats: int = 0) -> bool:
    count = Counter(filter(str.islower,path))
    return count["start"] == 1 and all([x <= 2 for x in count.values()]) and sum([x>1 for x in count.values()]) <= max_small_cave_repeats

assert     path_is_valid(["start"],0)
assert not path_is_valid(["start","a","a"],0)
assert     path_is_valid(["start","a","a","b"],1)
assert not path_is_valid(["start","a","a","b","b"],1)
assert not path_is_valid(['start', 'dc', 'DG', 'dc', 'DG', 'dc'],1)

def dfs(path, adj, max_small_cave_repeats):
    current = path[-1]
    if current == "end":
        return [path]
    else:
        out = []
        for k in adj[current]:
            new = [*path,k]
            if path_is_valid(new, max_small_cave_repeats):
                out += dfs(new, adj, max_small_cave_repeats)
        return out


In [49]:
def day12_1(input):
    paths = dfs(["start"],edges_to_adjacency_table(input),0)
    return len(paths)

def day12_2(input):
    paths = dfs(["start"],edges_to_adjacency_table(input),1)
    return len(paths)


In [50]:
do(12, 4241, 122134)

[4241, 122134]

### Day 13: Transparent Origami
Annoyingly this is not right and I haven't yet worked out what the problem is. TBC.

In [51]:
Point = NamedTuple("Point", [('x', int), ('y', int)])
Fold  = NamedTuple("Fold",  [('axis', int), ('v', int)])
Instructions = NamedTuple("Instructions", [('points',List[Point]),('folds',List[Fold])])

def parse_instructions(input: List[str]) -> Instructions:
    assert len(input) == 2
    points = [Point(*map(int,line.split(','))) for line in input[0].splitlines()]
    folds = [re.findall(r'(\w)=(\d+)',line)[0] for line in input[1].splitlines()]
    folds = [Fold(0 if ax=='x' else  1,int(v)) for ax,v in folds]
    return Instructions(points,folds)

in13: List[str] = data(13,sep="\n\n")
in13 = parse_instructions(in13)

In [52]:
def reflect(p: Point, fold: Fold):
    vals = list(p)
    vals[fold.axis] = vals[fold.axis] if vals[fold.axis] < fold.v else vals[fold.axis] - 2*(vals[fold.axis] - fold.v) # (fold.v - (vals[fold.axis] % fold.v)) % fold.v
    return Point(*vals)

def day13_1(instructions) -> int:
    points = instructions.points
    for fold in instructions.folds:
        points = [reflect(p,fold) for p in points] 
    return len(set(points))


In [53]:
do(13)

[104]

### Day 14: Extended Polymerization
Given an initial string (eg. _NNCB_) we are given a list of pair-insertion rules to extend the string. For example:
1. NN -> NCN
2. NC -> NBN

The hard bit is that these insertion rules need to be applied simultaneously and using over lapping windows, and the length of the string grows quite quickly. I initially tried to compute this directly (ie using the python _re.sub_ function to perform regex replacements), but this was too slow for part (II).

In [93]:
Instructions = NamedTuple("Instructions", [('template',str),('patterns',List[Tuple[str,str]])])

def parse_instructions(input: List[str]) -> Instructions:
    assert len(input) == 2
    patterns = [re.findall(r'(\w+) -> (\w+)',line)[0] for line in input[1].splitlines()] # eg. ('NN','B')
    return Instructions(input[0],patterns)

in14: List[str] = data(14,sep="\n\n")
in14 = parse_instructions(in14)

In [94]:

def mutate_polymer(template,rules,repeats):
    rules = {k:v for k,v in rules} # insertion rules
    counts = Counter([template[i:i+2] for i in range(len(template)-1)]) # counts of pairs of letter
    for j in range(repeats):
        new = Counter()
        for pair, count in counts.items():
            if pair in rules:
                new += Counter({pair[0]+rules[pair]:count, rules[pair]+pair[1]:count})
            else:
                new += Counter({pair:count})
        counts = new
    res = reduce(operator.add,[Counter({k[0]:v}) for k,v in counts.items()]) # single letter counts
    res += Counter(template[len(template)-1])
    return res

assert mutate_polymer('NNCB',[('NN','C'),('NC','B'),('CB','H')],1) == Counter('NCNBCHB')

def day14_1(instructions,repeats=10):
    counts = mutate_polymer(instructions.template,instructions.patterns,repeats)
    v = counts.most_common()
    return v[0][1] - v[-1][1]

def day14_2(instructions):  return day14_1(instructions,40)
        

In [96]:
do(14, 2010, 2437698971143)

[2010, 2437698971143]

### Day 16: Packet Decoder
'Packets' of information are provided in compressed form as a hexadecimal string. Broadly speaking there are two types of packets:
1. Literal packets; which contain (verson-number, packet-id, integer)
2. Operator packets; which contain (version-number, packet-id, sub-packet-meta-info, sub-packets)

Because operator packets contain sub-packets within them, this becomes a recursive structure which can be translated into basic mathetical expressions. To answer the question one must decode the hexadecimal string properly and then parse the recursive packet structure.

In [174]:
def hex_to_bin(x):
    n = len(x)
    x = bin(int(x,16))[2:]
    return '0'*(4*n - len(x)) + x # pad leading zeros

in16: str = data(16,hex_to_bin,sep="\n\n")

In [179]:
def parse_literal(y):
    i, isLast, value = 0, False, []
    while not isLast:
        isLast = y[i] == '0'
        value.append(y[i+1:i+5])
        i += 5
    value = int("".join(value),2)
    return value, i

def parse_operator(y):
    i, vals = 1, []
    if y[0] == '0': # 15 bits containing length of sub-packet
        l, i = int(y[i:i+15],2), 16
        while i < 16+l:
            [v,packetEnd] = parse_packet(y[i:])
            i += packetEnd
            vals.append(v)        
    elif y[0] == '1': # 11 bits representing number of sub-packets
        n, i = int(y[i:i+11],2), 12
        for _ in range(n):
            [v,packetEnd] = parse_packet(y[i:])
            i += packetEnd
            vals.append(v)
    else:
        raise Exception('Unexpected packet length ID %s',y[0])
    return vals, i

def parse_packet(y):
    version, id = int(y[:3],2), int(y[3:6],2)
    if id == 4:
        out, ix_end = parse_literal(y[6:])
    else:
        out, ix_end = parse_operator(y[6:])
    return (version, id, out), 6+ix_end

assert parse_packet(hex_to_bin('D2FE28'))[0]         == (6,4,2021)
assert parse_packet(hex_to_bin('38006F45291200'))[0] == (1, 6, [(6, 4, 10), (2, 4, 20)])
assert parse_packet(hex_to_bin('EE00D40C823060'))[0] == (7, 3, [(2, 4, 1), (4, 4, 2), (1, 4, 3)])

In [195]:
def sum_version_nums(packets):
    if type(packets[2]) is list:
        return packets[0] + sum(map(sum_version_nums,packets[2]))
    else:
        return packets[0]

def evaluate_packets(packets):
    assert len(packets) == 3
    id, subs = packets[1], packets[2]
    if id in [5,6,7]:
        assert len(subs) == 2
    if id == 0:   fn = operator.add
    elif id == 1: fn = operator.mul
    elif id == 2: fn = min
    elif id == 3: fn = max
    elif id == 4: # literal
        return packets[2]
    elif id == 5: fn = operator.gt
    elif id == 6: fn = operator.lt
    elif id == 7: fn = operator.eq
    else: raise Exception('Unexpected packet id %s',id)
    
    return reduce(fn,[evaluate_packets(x) for x in subs])

def day16_1(input):
    return sum_version_nums(parse_packet(input[0])[0])

def day16_2(input):
    packets = parse_packet(input[0])[0]
    return evaluate_packets(packets)


In [196]:
do(16, 925, 342997120375)

[925, 342997120375]